In [26]:
#博客來獨家
import requests
from bs4 import BeautifulSoup
import csv
import time
from datetime import datetime
import re  # 用於提取 bookID

# 基本的 URL 模板，其中 page 是頁碼
url_template = 'https://search.books.com.tw/search/query/cat/1/qsub/001/sort/5/v/0/spell/3/ms2/ms2_1/page/{}/key/%E5%8D%9A%E5%AE%A2%E4%BE%86%E7%8D%A8%E5%AE%B6'

# 用來存取數據的列表
books_data = []

# 設定出版日期的篩選條件
date_threshold = datetime.strptime('2024-07-01', '%Y-%m-%d')

# 迭代頁碼
page_num = 1

while True:
    # 生成當前頁碼的 URL
    url = url_template.format(page_num)
    print(f"正在抓取第 {page_num} 頁: {url}")

    # 發送請求並解析 HTML
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # 選擇所有書籍的條目
    books = soup.select('div.cntlisearch15 div.table-searchlist-wrap table tbody')

    # 如果當前頁沒有書籍條目，則表示已經抓完所有書籍
    if not books:
        print(f"第 {page_num} 頁無書籍，已經抓取完成")
        break

    # 逐一解析每本書籍的資訊
    for book in books:
        try:
            # 書名
            title = book.select_one('h4 a').text.strip()

            # 書籍鏈結
            book_link = 'https:' + book.select_one('h4 a')['href'].strip()

            # 提取 bookID (即 URL 中的數字部分)
            book_id = re.search(r'/products/(\d+)', book_link).group(1) if re.search(r'/products/(\d+)', book_link) else 'N/A'

            # 作者
            authors = ', '.join([a.text.strip() for a in book.select('ul.list-date li a[rel="go_author"]')])

            # 出版日期
            pub_date_str = book.select_one('ul.list-date li').text.split('出版日期:')[-1].strip()
            pub_date = datetime.strptime(pub_date_str, '%Y-%m-%d')

            # 如果出版日期在 2024 年 7 月 1 日之後，才抓取數據
            if pub_date > date_threshold:
                # 優惠價
                price_info = book.select_one('ul.list-nav li').text
                discount_price = price_info.split('折,')[-1].strip().replace('元', '').strip()

                # 將數據存入列表，包含 bookID
                books_data.append([book_id, title, authors, pub_date_str, discount_price, book_link])

        except Exception as e:
            print(f"解析書籍時發生錯誤: {e}")

    # 休眠一段時間，避免對伺服器發送過多請求
    time.sleep(2)

    # 進入下一頁
    page_num += 1

# 生成當天日期並加入到 CSV 文件名中
current_date = datetime.now().strftime('%m%d')
csv_file_path = rf'C:\Users\USER\Desktop\book_exclusive_data{current_date}.csv'  # 確保替換為正確路徑

# 將結果保存為 CSV 檔案，並新增 bookID 列
with open(csv_file_path, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['bookID', '書名', '作者', '出版日期', '優惠價', '鏈結'])
    writer.writerows(books_data)

print(f"書籍資訊已保存到 {csv_file_path}")



正在抓取第 1 頁: https://search.books.com.tw/search/query/cat/1/qsub/001/sort/5/v/0/spell/3/ms2/ms2_1/page/1/key/%E5%8D%9A%E5%AE%A2%E4%BE%86%E7%8D%A8%E5%AE%B6
正在抓取第 2 頁: https://search.books.com.tw/search/query/cat/1/qsub/001/sort/5/v/0/spell/3/ms2/ms2_1/page/2/key/%E5%8D%9A%E5%AE%A2%E4%BE%86%E7%8D%A8%E5%AE%B6
正在抓取第 3 頁: https://search.books.com.tw/search/query/cat/1/qsub/001/sort/5/v/0/spell/3/ms2/ms2_1/page/3/key/%E5%8D%9A%E5%AE%A2%E4%BE%86%E7%8D%A8%E5%AE%B6
正在抓取第 4 頁: https://search.books.com.tw/search/query/cat/1/qsub/001/sort/5/v/0/spell/3/ms2/ms2_1/page/4/key/%E5%8D%9A%E5%AE%A2%E4%BE%86%E7%8D%A8%E5%AE%B6
正在抓取第 5 頁: https://search.books.com.tw/search/query/cat/1/qsub/001/sort/5/v/0/spell/3/ms2/ms2_1/page/5/key/%E5%8D%9A%E5%AE%A2%E4%BE%86%E7%8D%A8%E5%AE%B6
正在抓取第 6 頁: https://search.books.com.tw/search/query/cat/1/qsub/001/sort/5/v/0/spell/3/ms2/ms2_1/page/6/key/%E5%8D%9A%E5%AE%A2%E4%BE%86%E7%8D%A8%E5%AE%B6
正在抓取第 7 頁: https://search.books.com.tw/search/query/cat/1/qsub/001/sort/5/v/0/spel

In [21]:
import csv

# CSV 文件的路徑
csv_file = r"C:\Users\USER\Desktop\book_exclusive_data0927.csv"

# 打開並讀取 CSV 文件
with open(csv_file, mode='r', encoding='utf-8') as file:
    csv_reader = csv.reader(file)
    
    # 讀取 CSV 文件的每一行
    for row in csv_reader:
        print(row)

['書名', '作者', '出版日期', '優惠價', '鏈結']
['能量精油芳療全書：人類圖9大身心系統 ╳ 科學實證天然精油應用，對症改善情緒、疼痛的調理處方【博客來獨家 淨化天使—萊姆精油】', '李依倩', '2024-10-10', '713', 'https://search.books.com.tw/redirect/move/key/%E5%8D%9A%E5%AE%A2%E4%BE%86%E7%8D%A8%E5%AE%B6/area/mid/item/0011001907/page/1/idx/1/cat/001/pdf/0/spell/3']
['喀邁拉空間(博客來獨家版)', '泰絲．格里森, 尤傳莉', '2024-10-09', '504', 'https://search.books.com.tw/redirect/move/key/%E5%8D%9A%E5%AE%A2%E4%BE%86%E7%8D%A8%E5%AE%B6/area/mid/item/0011001765/page/1/idx/2/cat/001/pdf/0/spell/3']
['梅子老師這一班4-6套書【博客來獨家】', '王文華, Yuan', '2024-10-07', '810', 'https://search.books.com.tw/redirect/move/key/%E5%8D%9A%E5%AE%A2%E4%BE%86%E7%8D%A8%E5%AE%B6/area/mid/item/0011002134/page/1/idx/3/cat/001/pdf/0/spell/3']
['卡內基 人性的弱點/人性的優點【博客來獨家套書】：全新譯本，連續10年《紐約時報》暢銷書排行榜!', '戴爾‧卡內基, 雲中軒', '2024-10-03', '504', 'https://search.books.com.tw/redirect/move/key/%E5%8D%9A%E5%AE%A2%E4%BE%86%E7%8D%A8%E5%AE%B6/area/mid/item/0011002127/page/1/idx/4/cat/001/pdf/0/spell/3']
['墨菲定律/彼得原理【博客來獨家套書】：《紐約時報》強力推薦，讓你讀懂生活與工作的真相!', '